In [1]:
from weldx import GenericSeries, Q_, U_
from xarray import DataArray

## Allowed variables (expressions only)

In [2]:
class AllowedVarsSeries(GenericSeries):
    _allowed_variables = ["a"]

In [29]:
try:
    AllowedVarsSeries("a*b")
except ValueError as e:
    print(e)

'b' is not allowed as an expression variable of class AllowedVarsSeries


In [30]:
AllowedVarsSeries("4*a");

## Required variables (expressions only)

In [5]:
class RequiredVarsSeries(GenericSeries):
    _required_variables = ["c"]

In [6]:
try:
    RequiredVarsSeries("a*b")
except ValueError as e:
    print(e)

RequiredVarsSeries requires expression variable 'c'.


In [7]:
RequiredVarsSeries("a*c");

## Preprocess coordinates

In [8]:
def handle_time(instance, time, time_ref):
    from weldx import Time
    time = Time(time, time_ref)
    
    return dict(t=time.as_quantity())

In [9]:
class PreprocessCoordsSeries(GenericSeries):
    _evaluation_preprocessor = handle_time

### Discrete

In [10]:
pcs_discrete = PreprocessCoordsSeries(DataArray(Q_([1,2,4],"m"), dims=["t"], coords={"t":[2,4,7]}))

In [11]:
pcs_result_discrete = pcs_discrete(time="2000-01-01 15:00:05", time_ref="2000-01-01 15:00:00")
print(f"time: {pcs_result_discrete.data_array.t.data}")
print(f"data: {pcs_result_discrete.data}")

time: [5.]
data: [2.6666666666666665] m


In [12]:
try:
    pcs_discrete(x=3)
except TypeError:
    print("invalid input")

invalid input


### Expression

In [13]:
pcs_expr = PreprocessCoordsSeries("a * t", parameters=dict(a="3m/s"))

In [14]:
pcs_result_expr = pcs_expr(time="2000-01-01 15:00:05", time_ref="2000-01-01 15:00:00")
print(f"time: {pcs_result_expr.data_array.t.data}")
print(f"data: {pcs_result_expr.data}")

time: [5.]
data: [15.0] m


In [15]:
try:
    pcs_expr(x=3)
except TypeError:
    print("invalid input")

invalid input


## Required dimensions

In [16]:
class RequiredDimsSeries(GenericSeries):
    _required_dimensions = ["c"]

###  Discrete

In [17]:
try:
    RequiredDimsSeries(Q_([[1,2], [3,4]], "m"), ["a", "b"])
except ValueError as e:
    print(e)

RequiredDimsSeries requires dimension 'c'.


In [18]:
RequiredDimsSeries(Q_([[1,2], [3,4]], "m"), ["a", "c"]);

### Expression

In [19]:
try:
    RequiredDimsSeries("a*b")
except ValueError as e:
    print(e)

RequiredDimsSeries requires dimension 'c'.


In [20]:
RequiredDimsSeries("a*b", dims=dict(b="c"));

In [21]:
RequiredDimsSeries("a*c"); 

In [22]:
RequiredDimsSeries("a*b", parameters=dict(a=(Q_([1,3], "m"), "c")));

## Required unit dimensionality

In [23]:
class RequiredUnitDimSeries(GenericSeries):
    _required_unit_dimensionality = U_("m").dimensionality

### Discrete

In [24]:
try:
    RequiredUnitDimSeries(Q_("2K"))
except ValueError as e:
    print(e)

RequiredUnitDimSeries requires its output unit to be of dimensionality '[length]' but it actually is '[temperature]'.


In [25]:
RequiredUnitDimSeries(Q_("2m"));

### Expression

In [26]:
try:
    RequiredUnitDimSeries("a*b", units=dict(a="K"))
except ValueError as e:
    print(e)

RequiredUnitDimSeries requires its output unit to be of dimensionality '[length]' but it actually is '[temperature]'.


In [27]:
RequiredUnitDimSeries("a*b", units=dict(a="m"));